# Validation of Gridpoint and Spectral LAM templates: data encoding

The objective is here to encode and decode the same data using a modified version of ECCODES and WGRIB2.

## environment

based on Eccodes 2.6.0 (branch feature/MeteoFrance_lamOMM) and a modified version of wgrib2 2.0.5

In [1]:
export TESTHOME=/home/yann.genin/lamOMM
export WGRIB2_PREFIX=$TESTHOME/wgrib2/grib2/wgrib2/
export GRIB_API_CMAKE_PREFIX=$TESTHOME/eccodes_build
export GRIB_API_PREFIX=$GRIB_API_CMAKE_PREFIX
export GRIB_API_SOURCE_PREFIX=$TESTHOME/eccodes-2.6.0-Source

In [2]:
export WGRIB2=$WGRIB2_PREFIX/wgrib2
export GRIB_DUMP=$GRIB_API_PREFIX/bin/grib_dump
export GRIB_DEFINITION_PATH=$GRIB_API_SOURCE_PREFIX/definitions
export GRIB_SAMPLES_PATH=$GRIB_API_SOURCE_PREFIX/samples
export LD_LIBRARY_PATH=$GRIB_API_PREFIX/lib

export ECCODES_DEBUG=0

## Grid point data

compilation of program using Eccodes

In [3]:
cat lam_gp.c

#include <stdlib.h>
#include <math.h>
#include <stdio.h>
#include <string.h>

#include "grib_api.h"

/*
 * Test encoding of LAM gridpoint fields
 * v1.0  philippe.marguinaud@meteo.fr, 02/2016
 */


int main (int argc, char * argv[])
{
  grib_handle * h;
  const char * f_text = argv[1];
  const char * f_g1   = argv[2];
  const char * f_g2   = argv[3];
  double * values = NULL;
  FILE * fp;
  size_t size, nlon, nlat, len;
  const void * buffer = NULL;
  int err = 0;
  int i;

  if (argc != 4)
    {
      printf ("%s: text grib1 grib2\n", argv[0]);
      return 0;
    }

  fp = fopen (f_g1, "r");
  GRIB_CHECK ((h = grib_handle_new_from_file (0, fp, &err)) == NULL, 0);
  fclose (fp);

  fp = fopen (f_text, "r");
  fscanf (fp, "%ld %ld\n", &nlon, &nlat);
  len = nlon * nlat;
  values = (double *)malloc (sizeof (double) * len);
  for (i = 0; i < len; i++)
    fscanf (fp, "%lf\n", &values[i]);
  fclose (fp);

  GRIB_CHECK (grib_set_double_array (h, "values", values, len), 0);
  
  fp = fopen 

In [4]:
cc -g -o lam_gp.x lam_gp.c -I$GRIB_API_PREFIX/include -L$GRIB_API_PREFIX/lib -leccodes -Wl,-rpath,$GRIB_API_PREFIX/lib

### Gridded data : fill data files with values

In [5]:
cp lam_gp.data lam_gp.txt
cp lam_gp.template  lam_gp.grib

Encoding data using Eccodes

In [6]:
./lam_gp.x lam_gp.txt lam_gp.grib lam_gp_encode=GRIB_API.grib

Encoding same data using Wgrib2

In [7]:
$WGRIB2  -import_text lam_gp.txt lam_gp.grib -set_scaling 0 -12  -grib_out lam_gp_encode=WGRIB2.grib

1:0:d=2009052800:TMP:0 hybrid level:3600 sec fcst:


In [8]:
ls -l lam_gp_encode=GRIB_API.grib lam_gp_encode=WGRIB2.grib

-rw-rw-r-- 1 yann.genin yann.genin 9116 28 nov.  14:00 'lam_gp_encode=GRIB_API.grib'
-rw-rw-r-- 1 yann.genin yann.genin 9116 28 nov.  14:00 'lam_gp_encode=WGRIB2.grib'


### Gridded data: comparison of encoding metadata (use of simple packing)

In [9]:
$GRIB_DUMP -O lam_gp_encode=GRIB_API.grib |awk '/SECTION_5/{f=1}/SECTION_6/{f=0}f'

======================   SECTION_5 ( length=21, padding=0 )    ======================
1-4       section5Length = 21
5         numberOfSection = 5
6-9       numberOfValues = 4096
10-11     dataRepresentationTemplateNumber = 0 [Grid point data - simple packing (grib2/tables/4/5.0.table) ]
12-15     referenceValue = 276.794
16-17     binaryScaleFactor = -12
18-19     decimalScaleFactor = 0
20        bitsPerValue = 16
21        typeOfOriginalFieldValues = 0 [Floating point (grib2/tables/4/5.1.table) ]


In [10]:
$WGRIB2 -Sec5 -packing -v lam_gp_encode=GRIB_API.grib 

1:0:Sec5 len=21 #defined data points=4096 Data Repr. Template=5.0:packing=grid point data - simple packing,s val=(276.794+i*2^-12)*10^0, i=0..65535 (#bits=16)


In [11]:
$GRIB_DUMP -O lam_gp_encode=WGRIB2.grib |awk '/SECTION_5/{f=1}/SECTION_6/{f=0}f'

======================   SECTION_5 ( length=21, padding=0 )    ======================
1-4       section5Length = 21
5         numberOfSection = 5
6-9       numberOfValues = 4096
10-11     dataRepresentationTemplateNumber = 0 [Grid point data - simple packing (grib2/tables/4/5.0.table) ]
12-15     referenceValue = 276.794
16-17     binaryScaleFactor = -12
18-19     decimalScaleFactor = 0
20        bitsPerValue = 16
21        typeOfOriginalFieldValues = 0 [Floating point (grib2/tables/4/5.1.table) ]


In [12]:
$WGRIB2 -Sec5 -packing -v lam_gp_encode=WGRIB2.grib 

1:0:Sec5 len=21 #defined data points=4096 Data Repr. Template=5.0:packing=grid point data - simple packing,s val=(276.794+i*2^-12)*10^0, i=0..65535 (#bits=16)


### Gridded data: dump values

In [13]:
$GRIB_DUMP -d lam_gp_encode=GRIB_API.grib | \
  perl -e ' local $/ = undef;  $x = <>; my ($length, $values) = ($x =~ m/\bvalues\((\d+)\)\s+=\s+{\s*(.*?)\s*}/goms); \
            print "$length 1\n"; my @values = split (m/\s*,\s*/o, $values); print join ("\n", @values, "") ' \
  > lam_gp_encode=GRIB_API_dump=GRIB_API.txt
$GRIB_DUMP -d lam_gp_encode=WGRIB2.grib | \
  perl -e ' local $/ = undef;  $x = <>; my ($length, $values) = ($x =~ m/\bvalues\((\d+)\)\s+=\s+{\s*(.*?)\s*}/goms); \
            print "$length 1\n"; my @values = split (m/\s*,\s*/o, $values); print join ("\n", @values, "") ' \
  > lam_gp_encode=WGRIB2_dump=GRIB_API.txt

In [14]:
$WGRIB2  -text lam_gp_encode=GRIB_API_dump=WGRIB2.txt -V -d 1 lam_gp_encode=GRIB_API.grib
$WGRIB2  -text lam_gp_encode=WGRIB2_dump=WGRIB2.txt -V -d 1 lam_gp_encode=WGRIB2.grib

1:0:vt=2009052801:0 hybrid level:3600 sec fcst:TMP Temperature [K]:
    ndata=4096:undef=0:mean=279.719:min=276.794:max=288
    grid_template=23:
	polar stereographic grid with LAM extension (U=[53,53],C=[8,8]): (64 x 64) input (null) output WE:SN res -1
	South pole lat1 67.937201 lon1 25.158021 latD 0.000000 lonV 0.000000 dx 2500.000000 m dy 2500.000000 m

1:0:vt=2009052801:0 hybrid level:3600 sec fcst:TMP Temperature [K]:
    ndata=4096:undef=0:mean=279.719:min=276.794:max=288
    grid_template=23:
	polar stereographic grid with LAM extension (U=[53,53],C=[8,8]): (64 x 64) input (null) output WE:SN res -1
	South pole lat1 67.937201 lon1 25.158021 latD 0.000000 lonV 0.000000 dx 2500.000000 m dy 2500.000000 m



### Gridded data: compare values encoded with same software, decoding with different software

In [15]:
./compareValuesInTextFile.sh lam_gp_encode=GRIB_API_dump=GRIB_API.txt lam_gp_encode=GRIB_API_dump=WGRIB2.txt
./compareValuesInTextFile.sh lam_gp_encode=WGRIB2_dump=GRIB_API.txt lam_gp_encode=WGRIB2_dump=WGRIB2.txt


average difference 0 maximum difference 0 number of values 4096
average difference 0 maximum difference 0 number of values 4096


### Gridded data: compare values encoded with different software, decoding with same software

In [16]:
./compareValuesInTextFile.sh lam_gp_encode=GRIB_API_dump=GRIB_API.txt lam_gp_encode=WGRIB2_dump=GRIB_API.txt
./compareValuesInTextFile.sh lam_gp_encode=GRIB_API_dump=WGRIB2.txt lam_gp_encode=WGRIB2_dump=WGRIB2.txt

average difference 0 maximum difference 0 number of values 4096
average difference 0 maximum difference 0 number of values 4096


## Spectral bi-fourier data

compilation of program using Eccodes

In [17]:
cat lam_bf.c

#include <stdlib.h>
#include <math.h>
#include <stdio.h>
#include <string.h>

#include "grib_api.h"

/*
 * Test encoding of LAM spectral fields
 * v1.0  philippe.marguinaud@meteo.fr, 02/2016
 * v1.01 yann.genin 11/2018 */


int main (int argc, char * argv[])
{
  grib_handle * h;
  const char * f_text = argv[1];
  const char * f_g1   = argv[2];
  const char * f_g2   = argv[3];
  double * values = NULL;
  FILE * fp;
  size_t size, len, dum;
  const void * buffer = NULL;
  int err = 0;
  int i;

  if (argc != 4)
    {
      printf ("%s: text grib1 grib2\n", argv[0]);
      return 0;
    }

  fp = fopen (f_g1, "r");
  GRIB_CHECK ((h = grib_handle_new_from_file (0, fp, &err)) == NULL, 0);
  fclose (fp);

  fp = fopen (f_text, "r");
  fscanf (fp, "%d %d\n", &len, &dum);
  values = (double *)malloc (sizeof (double) * len);
  for (i = 0; i < len; i++)
    fscanf (fp, "%lf\n", &values[i]);
  fclose (fp);

  GRIB_CHECK (grib_set_long (h, "bitsPerValue", 16), 0);

  GRIB_CHECK (grib_set_double_ar

In [18]:
cc -g -o lam_bf.x lam_bf.c -I$GRIB_API_PREFIX/include -L$GRIB_API_PREFIX/lib -leccodes -Wl,-rpath,$GRIB_API_PREFIX/lib

### Spectral data: fill data files with values

In [19]:
cp lam_bf.data.gz lam_bf.txt.gz
gunzip lam_bf.txt.gz
cp lam_bf.template  lam_bf.grib

Encoding data using Eccodes

In [20]:
./lam_bf.x lam_bf.txt lam_bf.grib lam_bf_encode=GRIB_API.grib

Encoding same data using Wgrib2

In [21]:
$WGRIB2  -set_grib_type bifourier -import_text lam_bf.txt lam_bf.grib -set_scaling -1 -13 -grib_out lam_bf_encode=WGRIB2.grib

1:0dec_scale=-1 binscale=-13 refval=-3.864928 s=8192.000000 d=0.100000
:d=2015050200:TMP:hybrid pressure level:42 hour fcst:


### Spectral data: comparison of encoding metadata 

In [22]:
ls -l lam_bf_encode=GRIB_API.grib lam_bf_encode=WGRIB2.grib

-rw-rw-r-- 1 yann.genin yann.genin 3641173 28 nov.  14:00 'lam_bf_encode=GRIB_API.grib'
-rw-rw-r-- 1 yann.genin yann.genin 3641173 28 nov.  14:00 'lam_bf_encode=WGRIB2.grib'


In [23]:
$GRIB_DUMP -O lam_bf_encode=GRIB_API.grib |awk '/SECTION_5/{f=1}/SECTION_6/{f=0}f'

======================   SECTION_5 ( length=31, padding=0 )    ======================
1-4       section5Length = 31
5         numberOfSection = 5
6-9       numberOfValues = 1735448
10-11     dataRepresentationTemplateNumber = 53 [Spherical harmonics bifourier data - complex packing (grib2/tables/4/5.0.table) ]
12-15     referenceValue = -3.86493
16-17     binaryScaleFactor = -13
18-19     decimalScaleFactor = -1
20        bitsPerValue = 16
21        biFourierSubTruncationType = 77 [Unknown code table entry () ]
22        biFourierDoNotPackAxes = 1
23-26     laplacianScalingFactor = 930404
27-28     biFourierResolutionSubSetParameterN = 106
29-30     biFourierResolutionSubSetParameterM = 106
31        unpackedSubsetPrecision = 2 [IEEE 64-bit  (I=8 in Section 7)  (grib2/tables/4/5.7.table) ]


In [24]:
$WGRIB2 -Sec5 -packing -v lam_bf_encode=GRIB_API.grib 

1:0:Sec5 len=31 #defined data points=1735448 Data Repr. Template=5.53:packing=spectral bifourier data - complex packing,_ val=(-3.86493+i*2^-13)*10^1, i=0..65535 (#bits=16) P-Laplacian scaling factor*10^-6=930404 Ns=106 Ms=106 SubTruncationType=77 doNotPackAxes=1 code_table_5.7=2


In [25]:
$GRIB_DUMP -O lam_bf_encode=WGRIB2.grib |awk '/SECTION_5/{f=1}/SECTION_6/{f=0}f'

======================   SECTION_5 ( length=31, padding=0 )    ======================
1-4       section5Length = 31
5         numberOfSection = 5
6-9       numberOfValues = 1735448
10-11     dataRepresentationTemplateNumber = 53 [Spherical harmonics bifourier data - complex packing (grib2/tables/4/5.0.table) ]
12-15     referenceValue = -3.86493
16-17     binaryScaleFactor = -13
18-19     decimalScaleFactor = -1
20        bitsPerValue = 16
21        biFourierSubTruncationType = 77 [Unknown code table entry () ]
22        biFourierDoNotPackAxes = 1
23-26     laplacianScalingFactor = 930403
27-28     biFourierResolutionSubSetParameterN = 106
29-30     biFourierResolutionSubSetParameterM = 106
31        unpackedSubsetPrecision = 2 [IEEE 64-bit  (I=8 in Section 7)  (grib2/tables/4/5.7.table) ]


In [26]:
$WGRIB2 -Sec5 -packing -v lam_bf_encode=WGRIB2.grib 

1:0:Sec5 len=31 #defined data points=1735448 Data Repr. Template=5.53:packing=spectral bifourier data - complex packing,_ val=(-3.86493+i*2^-13)*10^1, i=0..65535 (#bits=16) P-Laplacian scaling factor*10^-6=930403 Ns=106 Ms=106 SubTruncationType=77 doNotPackAxes=1 code_table_5.7=2


### Spectral data: dump values

In [27]:
$GRIB_DUMP -d lam_bf_encode=GRIB_API.grib | \
  perl -e ' local $/ = undef;  $x = <>; my ($length, $values) = ($x =~ m/\bvalues\((\d+)\)\s+=\s+{\s*(.*?)\s*}/goms); \
            print "$length 1\n"; my @values = split (m/\s*,\s*/o, $values); print join ("\n", @values, "") ' \
  > lam_bf_encode=GRIB_API_dump=GRIB_API.txt
$GRIB_DUMP -d lam_bf_encode=WGRIB2.grib | \
  perl -e ' local $/ = undef;  $x = <>; my ($length, $values) = ($x =~ m/\bvalues\((\d+)\)\s+=\s+{\s*(.*?)\s*}/goms); \
            print "$length 1\n"; my @values = split (m/\s*,\s*/o, $values); print join ("\n", @values, "") ' \
  > lam_bf_encode=WGRIB2_dump=GRIB_API.txt


In [28]:
$WGRIB2  -text lam_bf_encode=GRIB_API_dump=WGRIB2.txt -V -d 1 lam_bf_encode=GRIB_API.grib
$WGRIB2  -text lam_bf_encode=WGRIB2_dump=WGRIB2.txt -V -d 1 lam_bf_encode=WGRIB2.grib

1:0:vt=2015050318:hybrid pressure level:42 hour fcst:TMP Temperature [K]:
    ndata=1735448:undef=0:mean=0.000159003:min=-2.46804:max=280.596
    grid_template=63:Spectral Lambert 46700000 N=767 M=719, code_table_3.6=99 code_table_3.25=99
	Subdomains definition lx=1870700000 lux=1856400000 lcx=19500000 ly=1995500000 luy=1981200000 lcy=19500000
	South Pole latin1 46700000 latin2 46700000 latSP 0 lonSP 0

1:0:vt=2015050318:hybrid pressure level:42 hour fcst:TMP Temperature [K]:
    ndata=1735448:undef=0:mean=0.000159:min=-2.46804:max=280.596
    grid_template=63:Spectral Lambert 46700000 N=767 M=719, code_table_3.6=99 code_table_3.25=99
	Subdomains definition lx=1870700000 lux=1856400000 lcx=19500000 ly=1995500000 luy=1981200000 lcy=19500000
	South Pole latin1 46700000 latin2 46700000 latSP 0 lonSP 0



### Spectral data: compare values encoded with same software, decoding with different software

In [29]:
./compareValuesInTextFile.sh lam_bf_encode=GRIB_API_dump=GRIB_API.txt lam_bf_encode=GRIB_API_dump=WGRIB2.txt
./compareValuesInTextFile.sh lam_bf_encode=WGRIB2_dump=GRIB_API.txt lam_bf_encode=WGRIB2_dump=WGRIB2.txt

average difference 1.6055e-12 maximum difference 1e-08 number of values 1735448
average difference 1.67888e-12 maximum difference 1e-08 number of values 1735448


### Spectral data: compare values encoded with different software, decoding with same software

In [30]:
./compareValuesInTextFile.sh lam_bf_encode=GRIB_API_dump=GRIB_API.txt lam_bf_encode=WGRIB2_dump=GRIB_API.txt
./compareValuesInTextFile.sh lam_bf_encode=GRIB_API_dump=WGRIB2.txt lam_bf_encode=WGRIB2_dump=WGRIB2.txt

average difference 4.50397e-09 maximum difference 3.9e-07 number of values 1735448
average difference 4.50399e-09 maximum difference 3.9e-07 number of values 1735448


## Cleanup

In [31]:
rm -f *.txt *.grib lam_*.x